In [1]:
!nvidia-smi

Mon May  6 13:17:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:21:00.0  On |                  N/A |
| 46%   50C    P8              22W / 260W |     55MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
#del os.environ["CUDA_VISIBLE_DEVICES"]
import tensorflow as tf
import sys
sys.path.append("../train")
sys.path.append("..")
import tools
import main as m
import matplotlib.pyplot as plt
import argparse
import json
from tensorflow.keras import backend as K
%load_ext autoreload
%autoreload 2

In [3]:
def dice(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    inputs = K.flatten(y_true)
    targets = K.flatten(y_pred)
    intersection = K.sum(inputs * targets)
    dice = tf.divide(
        2.0 * intersection+1,
        K.sum(y_true) + K.sum(y_pred) + 1)

    return 1 - dice

def focal(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    inputs = K.flatten(y_true)
    targets = K.flatten(y_pred)
    gamma=2.0
    focal=-K.mean(inputs*tf.math.pow(1-targets, gamma)*tf.math.log(targets))
    return focal

def gen_dice_coef(y_true, y_pred, smooth=1e-7):
    '''
    Dice coefficient for num_classes labels (classes). Ignores background pixel label 0
    Pass to model as metric during compile statement
    '''
    y_true_f = K.flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.flatten(y_pred)
    intersect = K.sum(y_true_f * y_pred_f, axis=-1)
    denom = K.sum(y_true_f + y_pred_f, axis=-1)
    return K.mean((2. * intersect / (denom + smooth)))

# generalized dice coefficient as loss function
def gen_dice_coef_loss(y_true, y_pred):
    '''
    Dice loss to minimize. Pass to model as loss during compile statement
    '''
    return 1 - gen_dice_coef(y_true, y_pred)

def log_gen_dice_coef_loss(y_true, y_pred):
    return tf.math.log(2 - gen_dice_coef(y_true, y_pred))

def LogBinaryFocalCrossentropy(y_true, y_pred):
    FE = tf.keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha=0.2, reduction="sum")
    return tf.math.log(1+FE(y_true, y_pred))

def LogBinaryCrossentropy(y_true, y_pred):
    CE = tf.keras.losses.BinaryCrossentropy(reduction="sum")
    CE(y_true, y_pred)
    #return tf.math.log(1+CE(y_true, y_pred))

def SumLoss(loss1, loss2):
    def loss_summed(y_true, y_pred):
        return loss1(y_true, y_pred)+loss2(y_true, y_pred)*10
    return loss_summed
    

In [4]:
epsilon = 1e-5
smooth = 1

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.9
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 2
    return K.pow((1-pt_1), (1/gamma))

In [5]:
def main (args):
    with open(args.arhitecture) as f:
        arhitecture = json.load(f)
    if "0" in arhitecture.keys():
        arhitecture = arhitecture["0"]
    dataset_train = tf.data.TFRecordDataset([args.train_dataset_path])
    tfrecord_shape = tools.model.get_shape_of_quadratic_image_tfrecord(dataset_train)
    dataset_train = dataset_train.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
    dataset_train = dataset_train.shuffle(5*args.batch_size).batch(args.batch_size).prefetch(2)
    dataset_val = tf.data.TFRecordDataset([args.test_dataset_path])
    dataset_val = dataset_val.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
    dataset_val = dataset_val.batch(args.batch_size).prefetch(2)

    mirrored_strategy = tf.distribute.MirroredStrategy()
    FT = focal_tversky
    with mirrored_strategy.scope():
        model = tools.model.unet_model((128, 128, 1), arhitecture)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=args.start_lr), loss=FT,
                      metrics=["Precision", "Recall", tools.model.F1_Score()])
    earlystopping_kb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5*args.decay_lr_patience, verbose=1,
                                                        restore_best_weights=True)
    terminateonnan_kb = tf.keras.callbacks.TerminateOnNaN()
    reducelronplateau_kb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=args.decay_lr_rate,
                                                                patience=args.decay_lr_patience, verbose=1)
    try:
        results = model.fit(dataset_train, epochs=args.epochs, validation_data=dataset_val,
                            callbacks=[earlystopping_kb, terminateonnan_kb, reducelronplateau_kb], verbose=1)
    except KeyboardInterrupt:
        model.save(args.model_destination)
        print ("Model saved")
        return model
    model.save(args.model_destination)
    return model

In [6]:
def parse_arguments(args):
    """Parse command line arguments.
    Args:
        args (list): Command line arguments.
    Returns:
        args (Namespace): Parsed command line arguments.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument('--train_dataset_path', type=str,
                        default='../DATA/train1.tfrecord',
                        help='Path to training dataset.')
    parser.add_argument('--test_dataset_path', type=str,
                        default='../DATA/test1.tfrecord',
                        help='Path to test dataset.')
    parser.add_argument('--arhitecture', type=str,
                        default="../DATA/arhitecture_tuned.json",
                        help='Path to a JSON containing definition of an arhitecture.')
    parser.add_argument('--model_destination', type=str,
                        default="../DATA/Trained_model2",
                        help='Path where to save the model once trained.')
    parser.add_argument('--epochs', type=int,
                        default=128,
                        help='Number of epochs.')
    parser.add_argument('--batch_size', type=int,
                        default=128,
                        help='Batch size.')
    parser.add_argument('--class_balancing_alpha', type=float,
                        default=0.95,
                        help='How much to weight the positive class in the loss function.')
    parser.add_argument('--start_lr', type=float,
                        default=0.00005,
                        help='Initial learning rate.')
    parser.add_argument('--decay_lr_rate', type=float,
                        default=0.5,
                        help='Rate at which to decay the learning rate upon reaching the plateau.')
    parser.add_argument('--decay_lr_patience', type=float,
                        default=2,
                        help='Number of iteration to wait upon reaching the plataeau.')
    return parser.parse_args(args)

In [ ]:
args = parse_arguments([])
model=main(args)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

I0000 00:00:1715026683.006119   45824 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    126/Unknown - 47s 157ms/step - loss: 0.9979 - precision: 4.0506e-04 - recall: 0.9426 - f1_score: 8.6786e-04

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()